# 日本語GPT会話モデルで雑談対話モデルを学習する

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

c:\Users\daiki\miniconda3\envs\env_rinna\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
#モデル：RinnaのRLHFモデル
model_name = 'rinna/japanese-gpt-neox-3.6b-instruction-ppo'
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(model_name, load_in_8bit=True, device_map='auto')


===================================BUG REPORT===================================
The following directories listed in your path were found to be non-existent: {WindowsPath('C')}
The following directories listed in your path were found to be non-existent: {WindowsPath('vs/workbench/api/node/extensionHostProcess')}
The following directories listed in your path were found to be non-existent: {WindowsPath('file'), WindowsPath('/c%3A/Users/daiki/.vscode/extensions/ms-ceintl.vscode-language-pack-ja-1.84.2023110109/translations/extensions/vscode.json-language-features.i18n.json')}
The following directories listed in your path were found to be non-existent: {WindowsPath('/matplotlib_inline.backend_inline'), WindowsPath('module')}
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
The following directories listed in your path were found to be non-existent: {WindowsPath('/usr/local/cuda/lib64')}
DEBUG: Possible options found for libcudart.so: set(

RuntimeError: Failed to import transformers.integrations.bitsandbytes because of the following error (look up to see its traceback):

        CUDA Setup failed despite GPU being available. Please run the following command to get more information:

        python -m bitsandbytes

        Inspect the output of the command and see if you can locate CUDA libraries. You might need to add them
        to your LD_LIBRARY_PATH. If you suspect a bug, please take the information from python -m bitsandbytes
        and open an issue at: https://github.com/TimDettmers/bitsandbytes/issues

In [ ]:
# Rinnaとの会話
'''
注意
1. \nは<NL>に変換する
2. 発話プロンプト
    人間は「ユーザー」、Rinnaは「システム」とする
    話者の後に「:」をつける
    「:」の後には半角スペースを入れる
    発話テキストは半角スペースの後に入力する

    <例> ユーザー: こんにちは<NL>システム: こんにちは
'''
while True:
    text = input('入力どうぞ！！：')
    if text == 'exit':
        break
    else:
        text = text.replace('\n',  '<NL>')
        text = f'ユーザー: {text} <NL> システム: '
        token_ids = tokenizer.encode(text, add_special_tokens=False, return_tensors='pt')

        with torch.no_grad():
            output_ids = model.generate(token_ids.to(model.device), do_sample=True, max_new_tokens=128, temperature=0.7, 
                                        pad_token_id=tokenizer.eos_token_id, bos_token_id=tokenizer.bos_token_id, eos_token_id=tokenizer.eos_token_id)
        
        output = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1) :])
        output = output.replace('<NL>', '\n').replace('</s>', '')
        print(output)